In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
sys.path.insert(0, '../../')
#import Datanalytics as da 
from JKBio import TerraFunction as terra
%load_ext autoreload
%autoreload 2
from JKBio import Helper as h

import pickle
from taigapy import TaigaClient
tc = TaigaClient()
import numpy as np
import itertools

from bokeh.plotting import *
from bokeh.models import HoverTool
output_notebook()
import matplotlib.pyplot as plt
%load_ext rpy2.ipython
import seaborn as sns
import gseapy
from JKBio.helper import pyDESeq2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import AgglomerativeClustering

from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

## getting data

In [ ]:
! gsutil mv gs://transfer-amlproject/*MP7624* gs://transfer-amlproject/RNPv2/

In [ ]:
! gsutil -m cp -r gs://transfer-amlproject/RNPv3 gs://amlproject/RNA/

In [ ]:
! gsutil ls gs://amlproject/

In [ ]:
sampleset='RNPv3'

In [ ]:
terra.uploadFromFolder('amlproject','RNPv2/',
                       'broad-firecloud-ccle/hg38_RNAseq',samplesetname=sampleset,
                      fformat="fastqR1R2", sep='_MP7624')

## Processing

In [ ]:
wm = dm.WorkspaceManager('broad-firecloud-ccle/hg38_RNAseq')

In [ ]:
submission_id = wm.create_submission("star_v1-0_BETA_cfg", sampleset, 'sample_set',expression='this.samples')
terra.waitForSubmission('broad-firecloud-ccle/hg38_RNAseq', submission_id)

In [ ]:
submission_id = wm.create_submission("rsem_v1-0_BETA_cfg", 
                                      sampleset,'sample_set',expression='this.samples')
terra.waitForSubmission('broad-firecloud-ccle/hg38_RNAseq', submission_id)

In [ ]:
submission_id = wm.create_submission("rsem_aggregate_results_v1-0_BETA_cfg", 
                                         sampleset)
terra.waitForSubmission('broad-firecloud-ccle/hg38_RNAseq', submission_id)

In [ ]:
results = wm.get_sample_sets().loc[sampleset]
rsem_genes_expected_count = results['rsem_genes_expected_count']

In [ ]:
results

In [ ]:
mkdir ../../data/RNPv3

In [ ]:
! gsutil cp $rsem_genes_expected_count ../../data/RNPv3/

In [ ]:
file = '../../data/RNPv3/'+rsem_genes_expected_count.split('/')[-1]

In [ ]:
file

In [ ]:
! gunzip $file

## getting spike in scales

In [ ]:
dd

In [ ]:
! bwa index -a bwtsw ../data/ERCC92/ERCC92.fa

In [ ]:
! samtools faidx ../data/ERCC92/ERCC92.fa

In [ ]:
from JKBio import Helper as h

In [ ]:
! ../../TrimGalore-0.6.5/trim_galore

In [ ]:
ls -alh res

In [ ]:
fastqs = !ls res/*.sam

In [ ]:
h.parrun(['samtools sort '+i+' -@ 8 -o '+i.split('.')[0]+'.sorted.bam' for i in fastqs], cores=1, add=['rm '+i for i in fastqs])

In [ ]:
fastqs = !ls res/*.bam
fastqs = [v for i ,v in enumerate(fastqs) if i <30]
fastqs

In [ ]:
h.getSpikeInControlScales('../data/ERCC92/ERCC92.fa', fastq=list(fastqs), mapper='bwa', pairedEnd=True, cores=10, pathtosam='samtools', pathtotrim_galore='../../TrimGalore-0.6.5/trim_galore', pathtobwa='bwa',totrim=False, tomap=False, tofilter=True, results='res/', toremove=True)

In [ ]:
for f in h.grouped(fastqs, 2):
    print(f[0])

## Analysis

In [ ]:
file

In [ ]:
rsem_genes_expected_count = pd.read_csv(file[:-3], sep='\t')

In [ ]:
rsem_genes_expected_count = pd.read_csv("../../data/RNPv3/RNPv3.rsem_genes_expected_count.txt", sep='\t')

In [ ]:
data = rsem_genes_expected_count.drop("transcript_id(s)",1)

In [ ]:
data["gene_id"] = h.convertGenes(data['gene_id'])[0]

In [ ]:
data=data.set_index('gene_id')

In [ ]:
data

In [ ]:
rename = {"1": "mr120-MV411-RNP_IRF2BP2-r4",
"2": "mr121-MV411-RNP_IRF2BP2-r5",
"3": "mr122-MV411-RNP_IRF2BP2-r6",
"4": "mr123-MV411-RNP_IRF8-r4",
"5": "mr124-MV411-RNP_IRF8-r5",
"6": "mr125-MV411-RNP_IRF8-r6",
"7": "mr126-MV411-RNP_MEF2D-r4",
"8": "mr127-MV411-RNP_MEF2D-r5",
"9": "mr128-MV411-RNP_MEF2D-r6",
"10": "mr129-MV411-RNP_MYC-r4",
"11": "mr130-MV411-RNP_MYC-r5",
"12": "mr131-MV411-RNP_MYC-r6",
"13": "mr132-MV411-RNP_RUNX1-r4",
"14": "mr133-MV411-RNP_RUNX1-r5",
"15": "mr134-MV411-RNP_RUNX1-r6",
"16": "mr135-MV411-RNP_RUNX2-r4",
"17": "mr136-MV411-RNP_RUNX2-r5",
"18": "mr137-MV411-RNP_RUNX2-r6",
"19": "mr138-MV411-RNP_SPI1-r4",
"20": "mr139-MV411-RNP_SPI1-r5",
"21": "mr140-MV411-RNP_SPI1-r6",
"22": "mr141-MV411-RNP_ZMYND8-r4",
"23": "mr142-MV411-RNP_ZMYND8-r5",
"24": "mr143-MV411-RNP_ZMYND8-r6",
"25": "mr144-MV411-RNP_LMO2-r4",
"26": "mr145-MV411-RNP_LMO2-r5",
"27": "mr146-MV411-RNP_LMO2-r6",
"28": "mr147-MV411-RNP_LYL1-r4",
"29": "mr148-MV411-RNP_LYL1-r5",
"30": "mr149-MV411-RNP_LYL1-r6",
"31": "mr150-MV411-RNP_MAX-r4",
"32": "mr151-MV411-RNP_MAX-r5",
"33": "mr152-MV411-RNP_MAX-r6",
"34": "mr153-MV411-RNP_ZEB2-r4",
"35": "mr154-MV411-RNP_ZEB2-r5",
"36": "mr155-MV411-RNP_ZEB2-r6",
"37": "mr156-MV411-RNP_MEF2C-r4",
"38": "mr157-MV411-RNP_MEF2C-r5",
"39": "mr158-MV411-RNP_MEF2C-r6",
"40": "mr159-MV411-RNP_MEIS1-r4",
"41": "mr160-MV411-RNP_MEIS1-r5",
"42": "mr161-MV411-RNP_MEIS1-r6",
"43": "mr162-MV411-RNP_FLI1-r4",
"44": "mr163-MV411-RNP_FLI1-r5",
"45": "mr164-MV411-RNP_FLI1-r6",
"46": "mr165-MV411-RNP_ELF2-r4",
"47": "mr166-MV411-RNP_ELF2-r5",
"48": "mr167-MV411-RNP_ELF2-r6",
"49": "mr168-MV411-RNP_GFI1-r4",
"50": "mr169-MV411-RNP_GFI1-r5",
"51": "mr170-MV411-RNP_GFI1-r6",
"52": "mr171-MV411-RNP_IKZF1-r4",
"53": "mr172-MV411-RNP_IKZF1-r5",
"54": "mr173-MV411-RNP_IKZF1-r6",
"55": "mr174-MV411-RNP_CEBPA-r4",
"56": "mr175-MV411-RNP_CEBPA-r5",
"57": "mr176-MV411-RNP_CEBPA-r6",
"58": "mr177-MV411-RNP_MYB-r4",
"59": "mr178-MV411-RNP_MYB-r5",
"60": "mr179-MV411-RNP_MYB-r6",
"61": "mr180-MV411-RNP_MYBL2-r1",
"62": "mr181-MV411-RNP_MYBL2-r2",
"63": "mr182-MV411-RNP_MYBL2-r3",
"64": "mr183-MV411-RNP_HOXA9-r4",
"65": "mr184-MV411-RNP_HOXA9-r5",
"66": "mr185-MV411-RNP_HOXA9-r6",
"67": "mr186-MV411-RNP_AAVS1-r1",
"68": "mr187-MV411-RNP_AAVS1-r2",
"69": "mr188-MV411-RNP_AAVS1-r3",
"70": "mr189-MV411-RNP_SP1-r4",
"71": "mr190-MV411-RNP_SP1-r5",
"72": "mr191-MV411-RNP_SP1-r6",
"73": "mr192-MV411-RNP_SP1-r7"}

In [ ]:
data.columns

In [ ]:
data.columns = [rename[i] for i in data.columns]

In [ ]:
data

## pre processing 

filter some more

In [ ]:
toremove = np.argwhere(data.values.var(1)==0)
toremove.ravel()

In [ ]:
toremove.shape

In [ ]:
data = data.drop(data.iloc[toremove.ravel()].index,0)

In [ ]:
data.shape

In [ ]:
ERCC = data[~data.index.str.contains('ENSG00')]

In [ ]:
data = data[~data.index.str.contains('ERCC-')]

In [ ]:
ensg = data[data.index.str.contains('ENSG00')]

In [ ]:
data = data[~data.index.str.contains('ENSG00')]

renormalize the data

In [ ]:
len(ERCC)

## Getting the Core TF information

In [ ]:
ctf=pd.read_csv('../data/CTF.csv',header=None)[0].values.tolist()
ctf

## correlation filtering

## Making and running the dashboard

In [ ]:
%%R
library('erccdashboard')

In [ ]:
ERCC = ERCC.astype(int)

In [ ]:
ERCC['Feature'] = ERCC.index

In [ ]:
ERCC

In [ ]:
experiments = list(set([i.split('-')[2] for i in ERCC.columns[:-1]]))
experiments.remove("RNP_AAVS1")

In [ ]:
from rpy2.robjects.packages import importr
erccdashboard = importr('erccdashboard')

In [ ]:
#TODO: compute the mass from concentration
###################################################
### code chunk number 3: defineInputData
###################################################
%R datType = "count" # "count" for RNA-Seq data, "array" for microarray data
%R isNorm = False # flag to indicate if input expression measures are already
               # normalized, default is FALSE 
%R filenameRoot = "RNPv2" # user defined filename prefix for results files
%R sample2Name = "AAAVS1" # name for sample 2 in the experiment
%R erccmix = "RatioPair" # name of ERCC mixture design, "RatioPair" is default
%R erccdilution = 1/100 # dilution factor used for Ambion spike-in mixtures
%R spikeVol = 1 # volume (in microliters) of diluted spike-in mixture added to 
             #   total RNA mass
%R totalRNAmass = 0.500 # mass (in micrograms) of total RNA 
%R choseFDR = 0.1 # user defined false discovery rate (FDR), default is 0.05

In [ ]:
cols = list(ERCC.columns)
cols.sort()
for val in experiments:
    d = {}
    e=0
    d.update({
        'Feature':'Feature'
    })
    for i in cols:
        if val in i:
            e+=1
            d.update({i: val.split('_')[-1]+'_'+str(e)})
    d.update({
        'mr186-MV411-RNP_AAVS1-r1': 'AAAVS1_1',
        'mr187-MV411-RNP_AAVS1-r2': 'AAAVS1_2',
        'mr188-MV411-RNP_AAVS1-r3': 'AAAVS1_3'
    })
    a = ERCC[list(d.keys())].rename(columns=d)
    a.to_csv('../data/ERCC_estimation.csv', index=None)
    val = val.split('_')[-1]
    torm = 'RNPv2.'+val+'.AAAVS1.All.Pvals.csv'
    ! rm $torm 
    %R -i val print(val)
    %R print(sample2Name)
    %R a <- read.csv('../data/ERCC_estimation.csv')
    %R print(head(a))
    %R exDat = ''
    try:
        %R -i val exDat = runDashboard(datType=datType, isNorm = isNorm, exTable=a, filenameRoot=filenameRoot, sample1Name=val, sample2Name=sample2Name, erccmix=erccmix, erccdilution=erccdilution, spikeVol=spikeVol, totalRNAmass=totalRNAmass, choseFDR=choseFDR)
    except Warning:
        print("failed for "+val)
        continue
    except:
        print('worked for '+val)
    %R print(summary(exDat))
    %R grid.arrange(exDat$Figures$dynRangePlot)
    %R grid.arrange(exDat$Figures$rocPlot)
    %R grid.arrange(exDat$Figures$lodrERCCPlot)
    %R grid.arrange(exDat$Figures$maPlot)

In [ ]:
%matplotlib inline
ig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(data.corr(), 
            xticklabels=data.columns,
            yticklabels=data.columns, ax=ax)

In [ ]:
model = AgglomerativeClustering(n_clusters=15,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(data.corr())
ii = itertools.count(data.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]
sort = labels.argsort()

In [ ]:
%matplotlib inline

sns.clustermap(data.corr(), figsize=(20, 20))

plt.savefig('../data/RNPv2_cluster_count.pdf')

In [ ]:
data.sum().tolist()

In [ ]:
data.shape

## DESEQ ANALYSIS

In [ ]:
data

In [ ]:
experiments = list(set([i.split('-')[2] for i in data.columns[:-1]]))

In [ ]:
experiments.remove("RNP_AAVS1")

In [ ]:
data['gene_id'] = data.index

In [ ]:
experiments

In [ ]:
data.columns

In [ ]:
for val in experiments:  
    design = pd.DataFrame(index=data.columns[:-1], columns=['DMSO','Target'], 
                          data=np.array([[1 if 'RNP_AAVS1' in i else 0 for i in data.columns[:-1]],[1 if val in i else 0 for i in data.columns[:-1]]]).T)
    design.index = design.index.astype(str).str.replace('-','.')
    deseq = pyDESeq2.pyDESeq2(count_matrix=data, design_matrix = design, 
                              design_formula='~DMSO + Target', gene_column="gene_id")
    deseq.run_deseq()
    deseq.get_deseq_result()
    r = deseq.deseq_result
    r.pvalue = np.nan_to_num(np.array(r.pvalue), 1)
    r.log2FoldChange = np.nan_to_num(np.array(r.log2FoldChange), 0)
    results[val] = r

## CTF (volcano)

In [ ]:
results

In [ ]:
for val in experiments:
    a = h.volcano(results[val],tohighlight=ctf,title=val, maxvalue= 60, searchbox=True, minlogfold=0.5)
    try:
        show(a)
    except RuntimeError:
        show(a)

## any bias in the data

In [ ]:
datad = data

In [ ]:
data = data.drop(columns='mr129-MV411-RNP_MYC-r4')

In [ ]:
col = {v:i for i, v in enumerate(set([i.split('-')[2] for i in data.columns[:-1]]))}

In [ ]:
red = PCA(2).fit_transform(data[data.columns[:-1]].T)
h.scatter(red, labels=data.columns[:-1], radi=60000, colors=[col[i.split('-')[2]] for i in data.columns[:-1]])

In [ ]:
red = PCA(30).fit_transform(data[data.columns[:-1]].T)
red = TSNE(2,4).fit_transform(red)

In [ ]:
red.shape

mr129-MYC-r4 seems weird

In [ ]:
h.scatter(red, labels=data.columns[:-1], radi=10, colors=[col[i.split('-')[2]] for i in data.columns[:-1]])

In [ ]:
pca = PCA(20)
red = pca.fit_transform(data.T)

In [ ]:
pca.explained_variance_ratio_

## GSEA analysis

In [ ]:
data

In [ ]:
res = {}

In [ ]:
experiments

In [ ]:
for val in experiments:
    print(val)
    totest = data[[v for v in data.columns[:-1] if val in v or 'AAVS1' in v]]
    cls = ['Condition' if val in v else 'DMSO' for v in totest.columns]
    res[val] = gseapy.gsea(data=totest, gene_sets='WikiPathways_2013', 
                cls= cls, no_plot=False, processes=10)
    res[val].res2d['Term'] = [i for i in res[val].res2d.index]
    sns.barplot(data=res[val].res2d.iloc[:25], x="es", y="Term",
                hue_order="geneset_size").set_title(val)

In [ ]:
with open('../data/pathways/wikipathway_RNPv2', 'wb') as f:
    pickle.dump(res,f)

In [ ]:
with open('../data/pathways/wikipathway_RNPv2','rb') as f:
    res = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
for val in experiments:
    res[val].res2d['Term'] = [i[3:].split('WP')[0] for i in res[val].res2d['Term']]
    sns.barplot(data=res[val].res2d.iloc[:25], x="es", y="Term",
                hue_order="geneset_size").set_title(val)
    plt.show()

In [ ]:
a = set()
for k, val in res.items():
    a.update(set(val.res2d.index))
a = {i:[0]*len(res) for i in a}
for n,(k, val) in enumerate(res.items()):
    for i,v in val.res2d.iterrows():
        a[i][n] = v.es

In [ ]:
res = pd.DataFrame(a, index=res.keys())

In [ ]:
res.index = [i[3:].split('WP')[0] for i in res.index]

In [ ]:
res.index = [i.split('-')[2] for i in res.columns]

In [ ]:
sns.clustermap(figsize=(25,20), data=res,vmin=-1,vmax=1,xticklabels=res.columns, yticklabels=res.index)

In [ ]:
fig.savefig("enriched_terms.png")

In [ ]:
res = {}

In [ ]:
totest

In [ ]:
cls

In [ ]:
res.keys()

In [ ]:
for i, val in enumerate(experiments[4:]):
    print(val)
    if val in res:
        continue
    totest = data[[v for v in data.columns[:-1] if val in v or 'AAVS1' in v]]
    cls = ['Condition' if val in v else 'DMSO' for v in totest.columns]
    res[val] = gseapy.gsea(data=totest, gene_sets='GO_Biological_Process_2015', 
                cls= cls, no_plot=False, processes=6)
    res[val].res2d['Term'] = [i for i in res[val].res2d.index]
    plt.figure(i)
    sns.barplot(data=res[val].res2d.iloc[:25], x="es", y="Term",
                hue_order="geneset_size").set_title(val)

In [ ]:
with open('../data/pathways/GO_Biological_Process_2015_RNPv2', 'wb') as f:
    pickle.dump(res,f)

In [ ]:
with open('../data/pathways/GO_Biological_Process_2015_RNPv2','rb') as f:
    res = pickle.load(f)

creating matrices

In [ ]:
a = set()
for k, val in res.items():
    a.update(set(val.res2d.Term))
a = {i:[0]*len(res) for i in a}
for n,(k, val) in enumerate(res.items()):
    for i,v in val.res2d.iterrows():
        a[v.Term][n] = v.es
res = pd.DataFrame(a, index=res.keys())
fig, ax = plt.subplots(figsize=(20,15))
sns.heatmap(ax=ax,data=res)

In [ ]:
model = AgglomerativeClustering(n_clusters=8,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(res)
ii = itertools.count(res.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]

In [ ]:
sort = labels.argsort()

In [ ]:
a = h.plotCorrelationMatrix(res.values[sort],res.index[sort].tolist(),interactive=True,title="RNP2_bioproc_corr")

In [ ]:
cluster1= ['LMO2','LYL1','MAX','MEF2C']
cluster2=['GFI1','FLI1','MYB','IKZF1','ELF2','CEBPa','MEIS1']
cluster3=['IRF2BP2','MEF2C','CDK6','MEF2D','IRF8','BRD4','MYC']
cluster4= ['RUNX1','RUNX2','ZMYND8']

In [ ]:
res.loc[cluster2].mean().sort_values()

In [ ]:
'GO_Molecular_Function_2015',
'GeneSigDB',
'ENCODE_TF_ChIP-seq_2014',
#'Drug_Perturbations_from_GEO_2014',
'GO_Cellular_Component_2015',
'GO_Biological_Process_2015',
'PPI_Hub_Proteins',
'WikiPathways_2013',
'TF-LOF_Expression_from_GEO',
# msig db C2 C6 H http://software.broadinstitute.org/gsea/msigdb/annotate.jsp
# max's crc

## compare to the drop of CTF

In [ ]:
ctf = [
 'BRD4',
 'CDK6',
 'CEBPA',
 'ELF2',
 'FLI1',
 'GFI1',
 'IKZF1',
 'IRF2BP2',
 'IRF8',
 'LMO2',
 'LYL1',
 'MAX',
 'MEF2C',
 'MEF2D',
 'MEIS1',
 'MYB',
 'MYC',
 'RUNX1',
 'RUNX2',
 'SPI1',
 'ZEB2',
 'ZMYND8'
]

In [ ]:
deseq = pd.DataFrame()
for k, val in results.items():
    deseq[k] = val.log2FoldChange
deseq=deseq.T

In [ ]:
deseq

In [ ]:
a = plotCorrelationMatrix(a, deseq.index[sort].tolist(),interactive=True)

In [ ]:
ctf[11] = 'CEBPa'
ctf[]

In [ ]:
ctf

dropping ETV6 SP1  GSE1 LDB1

In [ ]:
deseq.loc[['MYC',
 'MYB',
 'SPI1',
 'RUNX1',
 'IRF2BP2',
 'FLI1',
 'ELF2',
 'ZEB2',
 'GFI1',
 'LMO2',
 'CEBPa',
 'MEF2D',
 'MEF2C',
 'IRF8',
 'MEIS1',
 'RUNX2',
 'RUNX2',
 'ZMYND8']]

In [ ]:
show(a)

In [ ]:
deseq_ctf = deseq.loc[['MYC',
 'MYB',
 'SPI1',
 'RUNX1',
 'IRF2BP2',
 'FLI1',
 'ELF2',
 'ZEB2',
 'GFI1',
 'LMO2',
 'CEBPa',
 'MEF2D',
 'MEF2C',
 'IRF8',
 'MEIS1',
 'RUNX2',
 'ZMYND8']]
model = AgglomerativeClustering(n_clusters=7,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(deseq_ctf)
ii = itertools.count(deseq_ctf.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]
sort = labels.argsort()
a = deseq_ctf.values[sort]
a = plotCorrelationMatrix(a, deseq_ctf.index[sort].tolist(),interactive=True)

In [ ]:
show(a)

In [ ]:
model = AgglomerativeClustering(n_clusters=7,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(deseq)
ii = itertools.count(deseq.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]
sort = labels.argsort()
a = deseq.values[sort]
a = plotCorrelationMatrix(a, deseq.index[sort].tolist(),interactive=True)

In [ ]:
show(a)

tsne, pca, clustering accross TF, CRC, most var genes, both ways.